# Part 2: Vectorized Computation using Pandas and GeoPandas

Pandas is based on numpy, therefore it provides vectorized computation as well. 

&rarr; [Pandas User Guide: Accelerated Operations](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#accelerated-operations)

## 1. Vectorization in Pandas

To take a closer look at vectorized computation using Pandas, we will take a look at __Sofia Heisler's repository [PyCon 2017: Optimizing Pandas Code for Performance](https://github.com/s-heisler/pycon2017-optimizing-pandas)__. This repo contains the material of her talk which she gave at the PyCon Conference 2017.

&rarr; Watch her talk on [YouTube](https://www.youtube.com/watch?v=HN5d490_KKk): I really recommend it (especially if you like panda GIFs), she explains the matter very well.

&rarr; Read her [blog post](https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6)

### What we will do

Conference talks and tutorials on GitHub are a great way to stay up-to-date with current developments in the scientific Python world and a great resource to learn. Therefore, we will:

1. Fork her GitHub Repository, so that we have our own copy of it.
2. Clone the copied GitHub Repository to our computer. 
3. Work through the notebook to learn about optimzing Pandas code using vectorization. 

### After working through the notebook, answer the following questions:

In [4]:
import pandas as pd
import numpy as np

We have a fake data set on population and area of a few cities. 

In [14]:
cities_df = pd.DataFrame({"population": np.random.randint(100000, 500000, 100), 
                  "area": np.random.randint(100, 1000, 100)})

1. Suppose you would like to calculate the population density (population / area). Why is it not good to use the methods `iterrows()` or `itercols()` and only slightly better to use the `apply()` method?

2. What would be a better way to calculate the population density?

3. Why would it be even faster when using the `.values` property for calulation?

4. Use the `.apply()` method to calculate the population density only for cities with a population > 200000. All other cities should get the value `np.nan`.

## 2. Vectorization in GeoPandas

### GeoPandas is great ... but still a bit slow

Geopandas make spatial analysis in Python a lot easier, but it has got a bottleneck: geometric operations are performed using shapely, which - as we have seen - is not the fastest. In addition, operations along a series of shapely objects cannot be vectorized in Python. 

Do you know why? 

__Answer:__

### But the GeoPandas developers also found a solution for this problem: Yet another package - PyGEOS

The PyGEOS packages allows vectorized geometric calculations based on the C library GEOS. It is pretty new and still under development.

&rarr; Take a look at the [PyGEOS User Guide](https://pygeos.readthedocs.io/en/latest/#)

### PyGEOS integration in GeoPandas
The support of PyGEOS in GeoPandas is already partly implemented. So if pygeos is installed in your python environment you can enable the pygeos support. __But beware, it is still in development phase!__

&rarr; Take a look at the [changes in the geopandas code](https://github.com/geopandas/geopandas/pull/1154/commits/e0658280a54e8f8ad1e9023952671553c756230a)

&rarr; Follow the development on GitHub: 
* [GeoPandas performance: optimizing vectorized operations](https://github.com/geopandas/geopandas/issues/430) 
* [Integrating pygeos in GeoPandas for vectorized array operations](https://github.com/geopandas/geopandas/issues/1155)

### Compare the shapely vs. pygeos GeoPandas

In order to use pygeos support in GeoPandas, you need to install pygeos. Unfortunately, there some conflicts with packages in our _advgeo_ environment. So in order to execute the following code you need to set up a new conda environment which has pygeos and geopandas installed. 

#### 1. Set up pygeos environment

Setup a new conda environment and install the packages pygeos, geopandas and jupyter. 


#### 2. Start the jupyter notebook in this new environment and open this notebook again. 

#### 3. Run the comparison

In [43]:
import geopandas as gpd

### Scenario: Buffering locations of the DWD temperature stations

As a case study we will perform a simple geometric operation: buffering the points of the DWD temperature measurement stations. 

In [44]:
file_path = "./data/DWD_temperature_shp/DWD_temperature.shp"

In [45]:
temp_df = gpd.read_file(file_path)

In [46]:
temp_df.head()

,ALIASNAME,Elevation,Lat,Lon,temperatur,geometry
0,AACHEN (WEWA),202.0,50.7833,6.0833,9.7,POINT (6.08330 50.78330)
1,AHRENSBURG-WULFSDORF,46.0,53.6500,10.2000,8.2,POINT (10.20000 53.65000)
2,ALBSTADT-BADKAP,759.0,48.2000,8.9667,7.3,POINT (8.96670 48.20000)
3,ALBSTADT-EBINGEN,712.0,48.2000,9.0333,6.4,POINT (9.03330 48.20000)
4,ALBSTADT-ONSTMETTINGEN,794.0,48.2667,9.0000,6.4,POINT (9.00000 48.26670)


In [47]:
type(temp_df.geometry[0])

shapely.geometry.point.Point

#### Buffering using shapely
In order to use shapely to perform the geometric operations, we need to disable the pygeos support of GeoPandas. 

In [49]:
gpd.options.use_pygeos = False

In [50]:
data_shapely = gpd.read_file(file_path)

In [51]:
data_shapely = data_shapely.to_crs(epsg="32632", inplace=False)

In [ ]:
data_shapely.to_crs(epsg="32632", inplace=True)

In [52]:
%%timeit
data_shapely.buffer(100)

42.7 ms ± 6.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Buffering using PyGEOS

In [53]:
gpd.options.use_pygeos = True

In [55]:
data_pygeos = gpd.read_file(file_path)

In [56]:
data_pygeos.to_crs(epsg="32632", inplace=True)

In [57]:
%%timeit
data_pygeos.buffer(100)

21 ms ± 1.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Bonus:
Since the temperature data is loaded already: calculate the mean, min, max of the temperature of all stations, as you have done in exercise D in the Geoscripting course. 

### Resources

[Introducing pygeos](https://caspervdw.github.io/Introducing-Pygeos/)

[PyGEOS Documentation](https://pygeos.readthedocs.io/en/latest/)
   
[Cythonize Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/enhancingperf.html)
